In [2]:
from graphdatascience import GraphDataScience
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

In [3]:
uri = "bolt://localhost:7687" 
user = "neo4j"  
password = "12345678"
driver = GraphDatabase.driver(uri=uri,auth=(user,password),database='neo4j')
session = driver.session(database='neo4j')

### Visualize the graph

In [3]:

result = session.run("CALL db.schema.visualization()")
w = GraphWidget(graph = result.graph())
w.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [4]:
session.run('''
    CALL gds.graph.project(
  	'movielens_projection',
  	['User', 'Movie'],
  	'RATED',
    {
        relationshipProperties: {
            rating: {
                property: 'rating',
                defaultValue: 0.0
        }
        }
    }
    );
''')

In [80]:
session.run('''
CALL gds.graph.project(
  'ratings',
  ['User','Movie'],
  {
    RATED: {
      orientation: 'UNDIRECTED',
      properties: 'rating'
    }
  }
)
''')

In [81]:
session.run('''
CALL gds.fastRP.mutate('ratings',
  {
    embeddingDimension: 256,
    randomSeed: 7474,
    mutateProperty: 'embedding',
    relationshipWeightProperty: 'rating',
    iterationWeights: [0.8, 1, 1, 1]
  }
)
YIELD nodePropertiesWritten
''')

In [82]:
result = session.run('''
    CALL gds.knn.write('ratings', {
    topK: 5,
    nodeProperties: ['embedding'],
    randomSeed: 42,
    concurrency: 1,
    sampleRate: 1.0,
    deltaThreshold: 0.0,
    writeRelationshipType: "SIMILAR",
    writeProperty: "score"
})
YIELD nodesCompared, relationshipsWritten, similarityDistribution
RETURN nodesCompared, relationshipsWritten, similarityDistribution.mean as meanSimilarity
''').to_df()
print(result)

   nodesCompared  relationshipsWritten  meanSimilarity
0          10440                 52200        0.950769


In [6]:
similar_user = session.run('''
    MATCH (n:User)-[r:SIMILAR]->(m:User)
    RETURN n.id as person1, m.id as person2, r.score as similarity
    ORDER BY similarity DESCENDING, person1, person2 limit 10
''').to_df()
print(similar_user)

   person1  person2  similarity
0  user126  user379    0.989860
1  user379  user126    0.989860
2  user242  user468    0.987524
3  user468  user242    0.987524
4  user130  user574    0.985089
5  user574  user130    0.985089
6  user379  user507    0.984774
7  user507  user379    0.984774
8  user150  user270    0.984757
9  user270  user150    0.984757


### Recommendation by user

In [23]:
query = '''
MATCH (u:User {id: 'user29'})
MATCH (u)-[r1:SIMILAR]->(similarUser)-[r2:RATED]->(m:Movie)<-[:IS_GENRE_OF]-(genre:Genre)
WHERE NOT EXISTS((u)-[:RATED]->(m))
RETURN u.id AS userId, m.title AS movieTitle, m.id AS movieId, COLLECT(DISTINCT genre.name) AS genres, COLLECT(DISTINCT similarUser.id) AS similarUserIds
LIMIT 10
'''
result = session.run(query)
print(result.to_df())
# widget = GraphWidget(graph = result.graph(), overview_enabled=True)
# widget.show()

   userId                                    movieTitle movieId  \
0  user29                      Clear and Present Danger     349   
1  user29  Harry Potter and the Deathly Hallows: Part 1   81834   
2  user29                                    Black Swan   81591   
3  user29                                   Crazy Heart   73023   
4  user29                                        Avatar   72998   
5  user29                                      Invictus   72733   
6  user29                              Blind Side, The    72641   
7  user29                                 Up in the Air   72011   
8  user29                                Serious Man, A   71464   
9  user29        Harry Potter and the Half-Blood Prince   69844   

                                   genres      similarUserIds  
0        [Action, Crime, Drama, Thriller]           [user590]  
1            [Action, Adventure, Fantasy]           [user590]  
2                       [Drama, Thriller]  [user590, user261]  
3     

#### visualize the graph

In [27]:
query = '''
MATCH (u:User {id: 'user29'})
MATCH (u)-[r1:SIMILAR]->(similarUser)-[r2:RATED]->(m:Movie)
WHERE NOT EXISTS((u)-[:RATED]->(m))
RETURN u, similarUser, m, r1, r2
LIMIT 10
'''
result = session.run(query)
widget = GraphWidget(graph = result.graph())
widget.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

### Recommendation by movie

In [25]:
query = '''
MATCH (targetMovie:Movie {id: '68954'})
MATCH (targetMovie)-[similarity:SIMILAR]-(similarMovie:Movie)<-[:IS_GENRE_OF]-(genre:Genre)
WHERE similarMovie.id <> targetMovie.id
RETURN DISTINCT similarMovie.id AS MovieId, COLLECT(DISTINCT genre.name) AS genres, similarMovie.title AS movieTitle , similarity.score AS score
ORDER BY similarity.score DESC
LIMIT 10
'''
result = session.run(query)
print(result.to_df())
# widget = GraphWidget(graph = result.graph())
# widget.show()

  MovieId                                             genres  \
0   60069            [Adventure, Animation, Romance, Sci-Fi]   
1   72998                        [Action, Adventure, Sci-Fi]   
2   59315                        [Action, Adventure, Sci-Fi]   
3   58559                             [Action, Crime, Drama]   
4   79132  [Action, Crime, Drama, Mystery, Sci-Fi, Thriller]   
5   78499            [Adventure, Animation, Comedy, Fantasy]   
6  115617                        [Action, Animation, Comedy]   
7   71264                               [Fantasy, Animation]   
8  111659                                [Adventure, Action]   
9   89039                           [Sci-Fi, Romance, Drama]   

                          movieTitle     score  
0                             WALL·E  0.979104  
1                             Avatar  0.975069  
2                           Iron Man  0.974797  
3                   Dark Knight, The  0.969951  
4                          Inception  0.968386  
5

#### Visualize the graph

In [1]:
query = '''
MATCH (targetMovie:Movie {id: '68954'})
MATCH (targetMovie)-[similarity:SIMILAR]-(similarMovie:Movie)<-[:IS_GENRE_OF]-(genre:Genre)
WHERE similarMovie.id <> targetMovie.id
RETURN DISTINCT targetMovie, similarMovie, similarity
ORDER BY similarity.score DESC
LIMIT 10
'''
result = session.run(query)
widget = GraphWidget(graph = result.graph())
widget.show()

NameError: name 'session' is not defined